# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [10]:
# import libraries
import re
import numpy as np
import pandas as pd
import nltk
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
import os
import sys


nltk.download('punkt','wordnet')
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'



[nltk_data] Downloading package punkt to wordnet...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# load data from database

database_name = 'data/DisasterResponse.db'
engine = create_engine(os.path.join(r'sqlite:///..',database_name))
df = pd.read_sql_table('disasterResponse',engine)
df.head()
                       



,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#Creating my X and y dataframes.
X = df['message']
y = df.iloc[:,4:]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    '''
    INPUT
    text - a string that will be tokenized
    
    OUTPUT
    Returns a tokenized version of the original string
    clean_tokens - tokenized version
    
    '''
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
#The chosen classifier here was Kneighbors, despite not expecting a good score from it, it will serve just fine as a baseline.
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

    # train classifier
pipeline.fit(X_train, y_train)

    # predict on test data
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
print(classification_report(y_test, y_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      5011
               request       0.75      0.32      0.45      1093
                 offer       0.00      0.00      0.00        22
           aid_related       0.71      0.32      0.44      2752
          medical_help       0.35      0.10      0.15       546
      medical_products       0.20      0.09      0.12       349
     search_and_rescue       0.33      0.01      0.02       179
              security       0.50      0.01      0.02       124
              military       0.33      0.02      0.04       198
                 water       0.49      0.19      0.27       403
                  food       0.54      0.24      0.34       730
               shelter       0.39      0.16      0.22       588
              clothing       0.60      0.06      0.10       105
                 money       0.46      0.04      0.07       156
        missing_people       0.50      

C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [6]:
clf_param = MultiOutputClassifier(KNeighborsClassifier())
clf_param.get_params().keys()

dict_keys(['estimator__algorithm', 'estimator__leaf_size', 'estimator__metric', 'estimator__metric_params', 'estimator__n_jobs', 'estimator__n_neighbors', 'estimator__p', 'estimator__weights', 'estimator', 'n_jobs'])

In [9]:
#This fit process takes over 20minutes, since the choice for kneighbors was used as a baseline, it serves most as an test, therefore keeping it to 3 variables.
parameters = {'clf__estimator__n_neighbors': [3, 5, 7]}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv_results = cv.fit(X_train, y_train)


In [10]:
cv_results.best_params_

{'clf__estimator__n_neighbors': 3}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
y_pred = cv_results.predict(X_test)

In [12]:
print(classification_report(y_test, y_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.87      5039
               request       0.88      0.11      0.20      1101
                 offer       0.00      0.00      0.00        23
           aid_related       0.77      0.05      0.10      2701
          medical_help       0.50      0.01      0.03       556
      medical_products       0.65      0.03      0.06       324
     search_and_rescue       0.00      0.00      0.00       173
              security       0.00      0.00      0.00       109
              military       0.00      0.00      0.00       219
                 water       0.69      0.06      0.11       424
                  food       0.75      0.09      0.16       725
               shelter       0.65      0.07      0.12       571
              clothing       0.62      0.05      0.10        93
                 money       0.43      0.02      0.04       147
        missing_people       0.00      

C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
#RandomForest is a very consolidated classifier so let us see how well it does here.
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
#SGD was the very well recommended for this and it has the extra benefit of having a much faster processing time
pipeline3 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SGDClassifier()))
        ])



In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
    # train classifier
pipeline2.fit(X_train, y_train)

    # predict on test data
y_pred = pipeline2.predict(X_test)
print(classification_report(y_test, y_pred, target_names=y_test.columns))


                        precision    recall  f1-score   support

               related       0.83      0.97      0.89      5091
               request       0.88      0.43      0.57      1115
                 offer       0.00      0.00      0.00        32
           aid_related       0.80      0.62      0.70      2783
          medical_help       0.69      0.05      0.10       542
      medical_products       0.79      0.04      0.08       356
     search_and_rescue       0.73      0.06      0.11       181
              security       1.00      0.01      0.01       133
              military       0.44      0.02      0.03       220
                 water       0.87      0.23      0.36       438
                  food       0.88      0.43      0.57       748
               shelter       0.88      0.22      0.35       608
              clothing       0.83      0.05      0.09       101
                 money       0.88      0.04      0.08       157
        missing_people       0.50      

C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
clf_param = MultiOutputClassifier(SGDClassifier())
clf_param.get_params().keys()

dict_keys(['estimator__alpha', 'estimator__average', 'estimator__class_weight', 'estimator__early_stopping', 'estimator__epsilon', 'estimator__eta0', 'estimator__fit_intercept', 'estimator__l1_ratio', 'estimator__learning_rate', 'estimator__loss', 'estimator__max_iter', 'estimator__n_iter_no_change', 'estimator__n_jobs', 'estimator__penalty', 'estimator__power_t', 'estimator__random_state', 'estimator__shuffle', 'estimator__tol', 'estimator__validation_fraction', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs'])

In [27]:
#The sgd classifier had the best results in terms of processing and final size, so I decided to improve it with the use of gridsearchcv

parameters = [{'vect__max_df': (0.5, 0.75, 1.0),
               'clf__estimator__alpha': (0.00001, 0.000001),
               'clf__estimator__penalty': ('l2', 'elasticnet')}]
cv = GridSearchCV(pipeline3, parameters)

cv_results_sgd = cv.fit(X_train, y_train)

y_pred = cv_results_sgd.predict(X_test)

print(classification_report(y_test, y_pred, target_names=y_test.columns))


# train classifier
#pipeline3.fit(X_train, y_train)

    # predict on test data
#y_pred = pipeline3.predict(X_test)
#print(classification_report(y_test, y_pred, target_names=y_test.columns))


                        precision    recall  f1-score   support

               related       0.87      0.90      0.88      5044
               request       0.71      0.62      0.66      1121
                 offer       0.00      0.00      0.00        29
           aid_related       0.71      0.68      0.69      2750
          medical_help       0.54      0.30      0.39       534
      medical_products       0.58      0.32      0.41       335
     search_and_rescue       0.57      0.24      0.34       181
              security       0.15      0.02      0.03       113
              military       0.58      0.36      0.45       218
                 water       0.72      0.61      0.66       420
                  food       0.79      0.77      0.78       721
               shelter       0.71      0.55      0.62       599
              clothing       0.85      0.53      0.65       100
                 money       0.52      0.29      0.38       139
        missing_people       0.44      

C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kikob\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

As a note, the chosen classifier has a considerate influence on the size of the pickle file. Since github only accepts files up to 100mb, I chose to save only pipeline3 as the best model for the app.

In [25]:
#pickle.dump(pipeline, open( 'classifierKN.pkl', 'wb' ) )
#over 100mb

In [26]:
#pickle.dump(pipeline2, open( 'classifierFR.pkl', 'wb' ) )
#over 800mb

In [27]:
pickle.dump(pipeline3, open( 'classifierSG.pkl', 'wb' ) )
#Just below 10mb, very good!

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.